In [42]:
import pandas as pd
from PyPDF2 import PdfReader
from pymongo import MongoClient
import pdfplumber
import re
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

Data retrival & excel file generation by combining the distorted tables

In [43]:
# Function to extract dimensions and sizes dynamically from the PDF
def extract_dimensions_with_sizes(pdf_path):
    dimensions = []
    headers = []  # To store the main and sub-headers dynamically
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                if len(table) > 0:
                    # Extract the headers (first row) and data rows
                    current_headers = table[0]
                    if not headers:  # Capture headers only once
                        headers = current_headers
                    for row in table[1:]:
                        if len(row) == len(headers):  # Ensure valid row
                            row_data = dict(zip(headers, row))
                            dimensions.append(row_data)
    return dimensions, headers

# Function to process and structure extracted data
def process_dimensions(dimensions, headers):
    processed_data = []

    for record in dimensions:
        processed_record = {}
        for header, value in record.items():
            if header and header.strip():
                processed_record[header.strip()] = value
        processed_data.append(processed_record)

    # Convert to DataFrame
    df = pd.DataFrame(processed_data)
    return df

# Function to save DataFrame to MongoDB
def save_to_mongodb(df, db_name, collection_name):
    client = MongoClient("mongodb://localhost:27017/")
    db = client[db_name]
    collection = db[collection_name]

    # Convert DataFrame to dictionary records and insert into MongoDB
    records = df.to_dict(orient='records')
    collection.insert_many(records)
    print(f"Data saved to MongoDB database '{db_name}', collection '{collection_name}'")

# Function to save DataFrame to Excel with formatting
def save_to_excel_formatted(df, output_file):
    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name="Sheet1")
        worksheet = writer.sheets["Sheet1"]

        # Set column widths for better readability
        for i, col in enumerate(df.columns):
            column_width = max(df[col].astype(str).map(len).max(), len(col))
            worksheet.set_column(i, i, column_width)

    print(f"Formatted data successfully saved to {output_file}")

# Main execution
def main():
    pdf_path = r"E:\git\Automated-Email-Parsing-and-Document-Generation\input_doc.pdf"  # Replace with your PDF file path
    output_excel_path = "output_formatted.xlsx"  # Replace with your desired output Excel file path
    db_name = "PDFData_sample"
    collection_name = "DynamicDimensionsNew"

    # Extract dimensions dynamically
    dimensions, headers = extract_dimensions_with_sizes(pdf_path)

    # Process and structure dimensions
    dataframe = process_dimensions(dimensions, headers)

    # Save to MongoDB
    save_to_mongodb(dataframe, db_name, collection_name)

    # Save to Excel with formatting
    save_to_excel_formatted(dataframe, output_excel_path)

if __name__ == "__main__":
    main()


Data saved to MongoDB database 'PDFData_sample', collection 'DynamicDimensionsNew'
Formatted data successfully saved to output_formatted.xlsx


SpecSheet generation and total calculation

In [44]:
import pdfplumber
import pandas as pd
from pymongo import MongoClient
import random

client = MongoClient("mongodb://localhost:27017/")  # Update with your MongoDB URI

db = client["PDFData_sample"]
collection = db["spec_sheet"]

def extract_spec_sheet_data(pdf_path):
    spec_sheet_data = []

    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            tables = page.extract_tables()
            for table in tables:
                # Identify the Spec Sheet table
                if "Spec Sheet:" in page.extract_text():
                    headers = table[0]  # First row is assumed to be headers
                    cleaned_headers = [str(header).strip() if header else f"Column_{i}" for i, header in enumerate(headers)]

                    for row in table[1:]:
                        if len(row) == len(cleaned_headers):
                            # Create a clean row dictionary with valid string keys
                            row_data = dict(zip(cleaned_headers, row))
                            spec_sheet_data.append(row_data)
    return spec_sheet_data

def save_spec_sheet_to_mongo(data):
    if data:
        collection.insert_many(data)
        print(f"{len(data)} records saved to MongoDB collection 'spec_sheet'.")
    else:
        print("No data to save to MongoDB.")

def save_to_excel(df, output_file):
    # Remove the '_id' column if it exists
    if '_id' in df.columns:
        df = df.drop(columns=['_id'])

    # Move the 'Qty' column to the end if it exists
    if 'Qty' in df.columns:
        qty_column = df.pop('Qty')
        df['Qty'] = qty_column

        # Convert 'Qty' to numeric, handling errors
        df['Qty'] = pd.to_numeric(df['Qty'], errors='coerce')

        # Add 'perrate' and 'total' columns
        df['perrate'] = [round(random.uniform(0.2, 0.8), 2) for _ in range(len(df))]  # Assign random values in range 0.2 to 0.8 with 2 decimals
        df['total'] = df['Qty'] * df['perrate']

        # Add a final row for the grand total
        grand_total_row = {
            'Placement': 'Grand Total',
            'Qty': None,
            'perrate': None,
            'total': df['total'].sum()
        }
        df = pd.concat([df, pd.DataFrame([grand_total_row])], ignore_index=True)

    # Write to Excel with formatting
    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        df.to_excel(writer, index=False, sheet_name="Sheet1")
        workbook = writer.book
        worksheet = writer.sheets["Sheet1"]

        # Format the 'Grand Total' row
        grand_total_format = workbook.add_format({'bold': True})
        grand_total_row_index = len(df)  # Last row index
        worksheet.set_row(grand_total_row_index, None, grand_total_format)

    print(f"Spec Sheet data successfully saved to {output_file}")

def main():
    pdf_path = "input_doc.pdf"  # Replace with your PDF path

    # Extract Spec Sheet data
    spec_sheet_data = extract_spec_sheet_data(pdf_path)

    # Save to MongoDB
    save_spec_sheet_to_mongo(spec_sheet_data)

    # Save to Excel
    if spec_sheet_data:
        df = pd.DataFrame(spec_sheet_data)
        output_file = "spec_sheet_output.xlsx"
        save_to_excel(df, output_file)

if __name__ == "__main__":
    main()


8 records saved to MongoDB collection 'spec_sheet'.
Spec Sheet data successfully saved to spec_sheet_output.xlsx


C:\Users\Asus\AppData\Local\Temp\ipykernel_3828\3904969852.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([grand_total_row])], ignore_index=True)


Budget document generation

In [45]:
"""import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

def generate_pdf_from_excel(excel_file, output_file):
    # Load data from Excel
    df = pd.read_excel(excel_file)

    # Round the 'total' column and add a rounded 'Grand Total' if applicable
    if 'total' in df.columns:
        df['total'] = df['total'].round(2)
        # Check if 'Grand Total' row already exists
        if not (df['Placement'] == 'Grand Total').any():
            grand_total = df['total'].sum().round(2)
            grand_total_row = {
                'Placement': 'Grand Total',
                'Composition': '-',
                'Qty': '-',
                'perrate': '-',
                'total': grand_total
            }
            df = pd.concat([df, pd.DataFrame([grand_total_row])], ignore_index=True)

    # Create the PDF document
    doc = SimpleDocTemplate(output_file, pagesize=letter)
    elements = []

    # Define styles
    styles = getSampleStyleSheet()
    title_style = styles["Heading1"]
    title_style.alignment = 1
    normal_style = styles["Normal"]

    # Add title
    elements.append(Paragraph("Spec Sheet", title_style))
    elements.append(Spacer(1, 12))

    # Convert DataFrame to table data
    table_data = [
        [Paragraph(str(col), normal_style) for col in df.columns]
    ] + [
        [Paragraph(str(cell), normal_style) for cell in row] for row in df.fillna("-").values.tolist()
    ]

    # Create the table with adjusted column widths
    table = Table(table_data, colWidths=[100, 200, 50, 50, 50])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0, 0), (-1, -1), 5),
        ('RIGHTPADDING', (0, 0), (-1, -1), 5),
    ]))

    elements.append(table)

    # Build the PDF
    doc.build(elements)
    print(f"PDF generated successfully: {output_file}")

if __name__ == "__main__":
    excel_file = "spec_sheet_output.xlsx"  # Replace with your Excel file path
    output_file = "spec_sheet_generated.pdf"  # Desired output PDF file path
    generate_pdf_from_excel(excel_file, output_file)"""

'import pandas as pd\nfrom reportlab.lib.pagesizes import letter\nfrom reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer\nfrom reportlab.lib import colors\nfrom reportlab.lib.styles import getSampleStyleSheet\n\ndef generate_pdf_from_excel(excel_file, output_file):\n    # Load data from Excel\n    df = pd.read_excel(excel_file)\n\n    # Round the \'total\' column and add a rounded \'Grand Total\' if applicable\n    if \'total\' in df.columns:\n        df[\'total\'] = df[\'total\'].round(2)\n        # Check if \'Grand Total\' row already exists\n        if not (df[\'Placement\'] == \'Grand Total\').any():\n            grand_total = df[\'total\'].sum().round(2)\n            grand_total_row = {\n                \'Placement\': \'Grand Total\',\n                \'Composition\': \'-\',\n                \'Qty\': \'-\',\n                \'perrate\': \'-\',\n                \'total\': grand_total\n            }\n            df = pd.concat([df, pd.DataFrame([grand

In [46]:
import pdfplumber
import pandas as pd
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

def generate_pdf_from_excel_and_properties(excel_file, pdf_file, input_pdf):
    # Load data from Excel
    df = pd.read_excel(excel_file)

    # Extract properties and image from the input PDF
    properties = {}
    image_path = None

    with pdfplumber.open(input_pdf) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                for line in lines:
                    if "Style number:" in line:
                        properties["Style number"] = line.split(":")[-1].strip()
                    elif "Style:" in line:
                        properties["Style"] = line.split(":")[-1].strip()
                    elif "Brand:" in line:
                        properties["Brand"] = line.split(":", 1)[-1].strip()
                    elif "Sizes:" in line:
                        properties["Sizes"] = line.split(":", 1)[-1].strip()
                    elif "Commodity:" in line:
                        properties["Commodity"] = line.split(":", 1)[-1].strip()
                    elif "E-mail:" in line:
                        properties["E-mail"] = line.split(":")[-1].strip()
                    elif "Care Address:" in line:
                        properties["Care Address"] = line.split(":", 1)[-1].strip()

            # Extract the first image if available
            if page.images:
                for img_meta in page.images:
                    if 'name' in img_meta:
                        image_path = f"temp_image_{img_meta['name']}.jpg"
                        with open(image_path, "wb") as img_file:
                            img_file.write(page.streams[img_meta['stream']].get_data())
                        break

    # Round the 'total' column and add a rounded 'Grand Total' if applicable
    if 'total' in df.columns:
        df['total'] = df['total'].round(2)
        # Check if 'Grand Total' row already exists
        if not (df['Placement'] == 'Grand Total').any():
            grand_total = df['total'].sum().round(2)
            grand_total_row = {
                'Placement': 'Grand Total',
                'Composition': '-',
                'Qty': '-',
                'perrate': '-',
                'total': grand_total
            }
            df = pd.concat([df, pd.DataFrame([grand_total_row])], ignore_index=True)

    # Create the PDF document
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    elements = []

    # Define styles
    styles = getSampleStyleSheet()
    title_style = styles["Heading1"]
    title_style.alignment = 1
    normal_style = styles["Normal"]
    heading_style = styles["Heading2"]

    # Add title
    elements.append(Paragraph("Costing Sheet", title_style))
    elements.append(Spacer(1, 12))

    # Add properties section
    elements.append(Paragraph("<b>Properties:</b>", heading_style))

    # Define the correct order for properties
    property_order = [
        "Style number",
        "Style",
        "Brand",
        "Sizes",
        "Commodity",
        "E-mail",
        "Care Address"
    ]

    for key in property_order:
        if key in properties:
            elements.append(Paragraph(f"<b>{key}:</b> {properties[key]}", normal_style))
            elements.append(Spacer(1, 6))

    # Add image if available
    if image_path:
        try:
            elements.append(Spacer(1, 12))
            elements.append(Image(image_path, width=150, height=200))
        except Exception as e:
            elements.append(Paragraph("<i>Image not available</i>", normal_style))

    elements.append(Spacer(1, 12))

    # Add Spec Sheet title
    elements.append(Paragraph("Spec Sheet:", heading_style))
    elements.append(Spacer(1, 12))

    # Convert DataFrame to table data
    table_data = [
        [Paragraph(str(col), normal_style) for col in df.columns]
    ] + [
        [Paragraph(str(cell), normal_style) for cell in row] for row in df.fillna("-").values.tolist()
    ]

    # Create the table with adjusted column widths
    table = Table(table_data, colWidths=[100, 200, 50, 50, 50])
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('LEFTPADDING', (0, 0), (-1, -1), 5),
        ('RIGHTPADDING', (0, 0), (-1, -1), 5),
    ]))

    elements.append(table)

    # Build the PDF
    doc.build(elements)
    print(f"PDF generated successfully: {pdf_file}")

if __name__ == "__main__":
    excel_file = "spec_sheet_output.xlsx"  # Replace with your Excel file path
    input_pdf = "input_doc.pdf"  # Replace with your input PDF path
    output_file = "spec_sheet_generated.pdf"  # Desired output PDF file path
    generate_pdf_from_excel_and_properties(excel_file, output_file, input_pdf)


PDF generated successfully: spec_sheet_generated.pdf
